In [44]:
%run runaway_functionsv3

In [65]:
cl = Cluster("Alessi_43")
ra = cl.runaways_all()
ra['Temp. Est'] = ra['Teff']
ra.write(f"./Clusters/{cl.name}/{cl.name}_runaways_all.tsv", format='ascii.ecsv', overwrite=True)

In [61]:
from astropy.wcs import WCS
from astropy.io import fits
from astropy import units as u
from astropy.table import Table
from astropy.time import Time
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion,PixCoord, PointSkyRegion, PointPixelRegion, LineSkyRegion, LinePixelRegion
%matplotlib qt
allrun = cl.runaways_all()[:]
allrun_coord_now = SkyCoord(ra=allrun['RA_ICRS_1'], 
                        dec=allrun['DE_ICRS_1'],
                        distance=allrun['rgeo'], 
                        pm_ra_cosdec=allrun['rmRA'],
                        pm_dec=allrun['rmDE'],
                        obstime=Time('J2000')+500*u.kyr)

allrun_coord_earlier = allrun_coord_now.apply_space_motion(dt=-100*u.kyr)


with fits.open('./Clusters/Basel_8/Basel_8_extra10pc.fits') as fits_file:
    image = fits_file[0]
    wcs = WCS(image.header)
    fig, ax = plt.subplots(subplot_kw={'projection': wcs}, figsize=(8, 8))
    ax.imshow(image.data, 
              cmap='gray')
    ax.set_xlabel('Right Ascension (hms)')
    ax.set_ylabel('Declination (degrees)')

    c = SkyCoord("98.5582 +8.0381", distance=1456, unit=(u.deg ,u.deg, u.pc))
    radius = Angle(0.3, 'deg')
    region = CircleSkyRegion(c,radius)
    region_pix = region.to_pixel(wcs)
    region_pix.plot(ax=ax, 
                    color='red', 
                    lw=2)
    
    start_skys = allrun_coord_now
    end_skys = allrun_coord_earlier

    region_sky = [LineSkyRegion(start=start_sky, end=end_sky) for start_sky,end_sky in zip(allrun_coord_now,allrun_coord_earlier)]
    region_sky_pix = [region_sky_.to_pixel(wcs) for region_sky_ in region_sky]
    for c in allrun_coord_now:
        point_sky = PointSkyRegion(center=c)
        point_pix = point_sky.to_pixel(wcs)
        point_pix.plot(ax=ax,
                       color='yellow')
    for region_pix in region_sky_pix:
        region_pix.plot(ax=ax,
                        color='cyan')

/usr/lib/python3/dist-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [37]:
dias2021 = Table.read("dias2021.tsv", format="ascii.ecsv")
maskplx = dias2021['Plx'] > 0.3
maskage = dias2021['logage'] < 7.7
workclusters = []
for clustername in dias2021[maskplx & maskage][:]['Cluster']:
    if clustername not in ['ASCC_79','BH_164','BH_23','Collinder_135','Collinder_140','Gulliver_9','IC_2391','IC_2602','Mamajek_1','Platais_8','UPK_535','UPK_606','UPK_640','Berkeley_59','COIN-Gaia_37','Ivanov_4','LP_1937','Sigma_Ori','UBC_632']:
        workclusters.append(clustername)
        # print(clustername)
        # cl = Cluster(clustername)
        # cl.prepare_trace()
# cl.stars_in_region()
# cl.stars_in_region()

In [ ]:
workclusters_mask = [cluster in workclusters for cluster in dias2021['Cluster']]
workclusterstable = dias2021[workclusters_mask]

rvexists = workclusterstable['RV'].mask
nrvgood = workclusterstable['NRV'] > 4

workclusterstable[~rvexists & nrvgood]

In [68]:
def runaways_from(clustername):
    cl = Cluster(clustername)
    fs4giesler = cl.fast_stars_in_region()
    outputs = os.listdir(f"/home/surodeep/suro_aiu/traceback/cluster_runaway/{cl.name}/runaways/")
    linenos = []
    for output in outputs:
        #print(output)
        if 'run' in output:
            linenos.append(int(output.split("+")[1].replace(".out","")))
    linenos.sort()
    # print(linenos)
    i=np.array(linenos)-3
    def source_of(lineno, input_table):
        return input_table[lineno-2]['Source']
    return fs4giesler[i]
for cluster in workclusters:
    t = runaways_from(cluster)[runaways_from(cluster)['Teff'].argmax()]['Teff']
    
    if t>10000:
        print(cluster, t,runaways_from(cluster)[runaways_from(cluster)['Teff'].argmax()]['Source'])

Alessi_43 15014.8 5524355337493304832
Alessi_Teutsch_5 11066.8 2203047987818327936
Antalova_2 10476.0 4054754516779374592
BH_121 15232.7 5333952049933787520
BH_205 10292.0 5966670134990392320
BH_217 15042.6 5971838015762732544
Barkhatova_1 10381.6 2163399839041410560
Berkeley_79 15050.9 4259983000670960000
Bica_2 15004.1 2067785002204179200
Biurakan_2 15007.7 2058844701379362560
Bochum_10 10187.9 5350570927670271744
Bochum_11 10471.4 5350284711031815552
Bochum_13 15008.0 5975514026711634048
COIN-Gaia_41 10437.2 3350663730331408256
Collinder_104 11602.2 3130880461270803712
Collinder_107 11602.2 3130880461270803712
Collinder_272 10696.6 5868468770138838144
Czernik_31 10373.1 5715451523558459520
Czernik_41 15018.4 2026718410407452672
Dias_5 11380.4 4094456885225767680
Dolidze_53 15586.9 2020850321173536128
ESO_332_08 10077.2 5969667678565148160
ESO_332_13 10292.0 5966670134990392320
FSR_0165 15005.8 2027292737751016320
FSR_0198 15054.8 2059460805856016128
FSR_0451 10323.8 2011918575888660

ValueError: attempt to get argmax of an empty sequence